In [ ]:
%pylab inline

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


Connect to google drive

In [ ]:
import numpy as np
import pandas as pd
import sys
sys.path.append('ideal')
from IdealDataInterface import IdealDataInterface

### Read the ideal data from the folder

In [ ]:
folder_path = 'ideal/'
ideal = IdealDataInterface(folder_path)

Found entries for 219 sensor readings.


In [ ]:
from activations.calculate_activations import convert_activations_to_wall_time
from activations.calculate_activations import get_appliance_activations
from activations.calculate_activations import normalise_activations
from activations.calculate_daily_activations import get_daily_activations

from activations.calculate_daily_activations import detect_daily_appliance_activations
from activations.calculate_activations import get_appliance_activations



Found entries for 219 sensor readings.


### Get the data

In [ ]:
data=ideal.get()

In [ ]:
len(data)

219

In [ ]:
data[0]

{'homeid': '106',
 'roomid': '1085',
 'room_type': 'kitchen',
 'category': 'electric-appliance',
 'subtype': 'microwave',
 'sensorid': '5211',
 'readings': time
 2017-05-17 18:46:01      0
 2017-05-17 18:46:02      0
 2017-05-17 18:47:53      0
 2017-05-17 18:47:54      0
 2017-05-17 18:47:55    246
                       ... 
 2018-06-14 07:28:06      0
 2018-06-14 08:28:06      0
 2018-06-14 09:28:06      0
 2018-06-14 10:28:06      0
 2018-06-14 11:28:06      0
 Name: microwave, Length: 10414, dtype: int64}

In [ ]:
ideal.view(subtype='washingmachine')

,homeid,roomid,room_type,category,subtype,sensorid
0,73,781,kitchenliving,electric-appliance,washingmachine,4078
1,105,1110,kitchen,electric-appliance,washingmachine,5285
2,106,1085,kitchen,electric-appliance,washingmachine,5212
3,136,1294,kitchen,electric-appliance,washingmachine,9471
4,128,1253,kitchen,electric-appliance,washingmachine,13307
5,139,1315,kitchen,electric-appliance,washingmachine,13306
6,175,1636,diningroom,electric-appliance,washingmachine,11523
7,171,1565,kitchen,electric-appliance,washingmachine,11687
8,145,1349,kitchen,electric-appliance,washingmachine,14222
9,208,1935,kitchen,electric-appliance,washingmachine,18021


In [ ]:
ideal.view(homeid=105, subtype='sink')

/content/drive/MyDrive/ideal/IdealDataInterface.py:116: FutureWarning: The behavior of indexing on a MultiIndex with a nested sequence of labels is deprecated and will change in a future version. `series.loc[label, sequence]` will raise if any members of 'sequence' or not present in the index's second level. To retain the old behavior, use `series.index.isin(sequence, level=1)`
  return self.sensorid_mapping.loc(axis=0)[homeid, roomid, room_type, category, subtype, sensorid, filename].to_frame().T
/content/drive/MyDrive/ideal/IdealDataInterface.py:118: FutureWarning: The behavior of indexing on a MultiIndex with a nested sequence of labels is deprecated and will change in a future version. `series.loc[label, sequence]` will raise if any members of 'sequence' or not present in the index's second level. To retain the old behavior, use `series.index.isin(sequence, level=1)`
  return self.sensorid_mapping.loc(axis=0)[homeid, roomid, room_type, category, subtype, sensorid, filename]


,homeid,roomid,room_type,category,subtype,sensorid


### Sketch the agile tariff data

In [ ]:
df = pd.read_csv('csv_agile_N_Southern_Scotland.csv')

In [ ]:
df.columns
new_column_names = ['time', 'Time', 'N', 'region','price']  # List of new column names

# Use the 'rename' method to change the column names
df = df.rename(columns=dict(zip(df.columns, new_column_names)))

In [ ]:
df

,time,Time,N,region,price
0,2018-02-21 00:30:00+00:00,00:30,N,Southern_Scotland,10.5735
1,2018-02-21 01:00:00+00:00,01:00,N,Southern_Scotland,10.6680
2,2018-02-21 01:30:00+00:00,01:30,N,Southern_Scotland,10.1850
3,2018-02-21 02:00:00+00:00,02:00,N,Southern_Scotland,10.1430
4,2018-02-21 02:30:00+00:00,02:30,N,Southern_Scotland,10.1955
...,...,...,...,...,...
92582,2023-06-03 19:30:00+00:00,20:30,N,Southern_Scotland,16.3380
92583,2023-06-03 20:00:00+00:00,21:00,N,Southern_Scotland,16.4010
92584,2023-06-03 20:30:00+00:00,21:30,N,Southern_Scotland,16.0965
92585,2023-06-03 21:00:00+00:00,22:00,N,Southern_Scotland,18.1230


In [ ]:
def lowest(date):

    # Convert the 'time' column to datetime type if it's not already in datetime format
    df['time'] = pd.to_datetime(df['time'])
    date_to_check = pd.to_datetime(date).date()  # Convert date to datetime.date object

    # Filter the DataFrame for the specified date
    filtered_df = df[df['time'].dt.date == date_to_check]

    # Get the row with the lowest price
    lowest_price_row = filtered_df.nsmallest(1, 'price')

    # Return the lowest price
    return lowest_price_row['price'].values[0]








In [ ]:
def lowestRange(date, k):

    # Convert the 'time' column to datetime type if it's not already in datetime format
    df['time'] = pd.to_datetime(df['time'])
    date_to_check = pd.to_datetime(date).date()  # Convert date to datetime.date object

    # Filter the DataFrame for the specified date
    filtered_df = df[df['time'].dt.date == date_to_check]

    rolling_mean = filtered_df['price'].rolling(window=k).mean()



    return np.min(rolling_mean)



In [ ]:
def round_down_to_half_hour(timestamp):
    # Subtract the minutes modulo 30 to round down to the nearest half-hour
    rounded_timestamp = timestamp - pd.to_timedelta(timestamp.minute % 30, unit='m')
    # Set the seconds and microseconds to 0 to get the result without minutes and seconds
    rounded_timestamp = rounded_timestamp.replace(second=0, microsecond=0)
    return rounded_timestamp


In [ ]:

def getPrice(time):
# Assuming 'df' is the DataFrame containing the data
# Assuming 'desired_time' is the time you want to round
  desired_time = time

  # Convert the 'desired_time' to a pandas Timestamp object
  desired_timestamp = pd.to_datetime(desired_time)

  # Round the timestamp to the nearest hour using the floor() method
  rounded_timestamp = round_down_to_half_hour(desired_timestamp)

  # Find the row in the DataFrame that matches the rounded timestamp
  row = df[df['time'] == str(rounded_timestamp)+'+00:00']

#   if not row.empty():

  # Get the corresponding value from the DataFrame
  corresponding_value = row['price'].values[0]
#   else:
#     return None

  return corresponding_value


In [ ]:
getPrice('2020-12-01 19:01:29')

12.1695

In [ ]:
import csv
with open('savings 2022.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['homeid', 'roomid','appliancetype', 'times used', 'years','savings(pences)'])


**Add hypothesis:**
## **1. can not be shifted: microwave, fridgefreezer, electricheater, freezer, toaster, fridge, grill, electricoven, appliance, other**
# **2. can be shifted to anytime of tha day: kettle, dishwasher, washingmachine, dehumidifier, washingmachinetumbledrier, tumbledrier**
# **3. whose data is not easy to record and not valid for modeling: vacuum cleaner**


In [ ]:
ideal.view(subtype = ['electric'])

/content/drive/MyDrive/ideal/IdealDataInterface.py:116: FutureWarning: The behavior of indexing on a MultiIndex with a nested sequence of labels is deprecated and will change in a future version. `series.loc[label, sequence]` will raise if any members of 'sequence' or not present in the index's second level. To retain the old behavior, use `series.index.isin(sequence, level=1)`
  return self.sensorid_mapping.loc(axis=0)[homeid, roomid, room_type, category, subtype, sensorid, filename].to_frame().T
/content/drive/MyDrive/ideal/IdealDataInterface.py:118: FutureWarning: The behavior of indexing on a MultiIndex with a nested sequence of labels is deprecated and will change in a future version. `series.loc[label, sequence]` will raise if any members of 'sequence' or not present in the index's second level. To retain the old behavior, use `series.index.isin(sequence, level=1)`
  return self.sensorid_mapping.loc(axis=0)[homeid, roomid, room_type, category, subtype, sensorid, filename]


,homeid,roomid,room_type,category,subtype,sensorid


In [ ]:
data1=ideal.get(subtype = ['kettle','washingmachine', 'dehumidifier', 'washingmachinetumbledrier', 'tumbledrier'])
# data1=ideal.get(subtype = ['dishwasher'])

In [ ]:
len(data1)

20

In [ ]:
import pandas as pd

def create_lists_within_time_range(df, dk):
    result_lists1 = []
    result_lists2 = []

    for _, row in dk.iterrows():
        start_time = row['start']
        end_time = row['end']
        current_list1 = []
        current_list2 = []
        in_range = False

        for timestamp, val in zip(df.index, df.values):
            if not in_range and abs((timestamp - start_time).total_seconds()) < 30:
                in_range = True

            if in_range:
                current_list1.append(timestamp)
                current_list2.append(val)

            if in_range and abs((timestamp - end_time).total_seconds()) < 30:
                if len(current_list1) > 0 and len(current_list2) > 0:
                    result_lists1.append(current_list1)
                    result_lists2.append(current_list2)
                current_list1 = []
                current_list2 = []
                in_range = False
                break

    return result_lists1, result_lists2

In [ ]:
import pandas as pd

def calculate_hours(start_timestamp, end_timestamp):
    time_diff = end_timestamp - start_timestamp
    hours = time_diff.total_seconds() / 3600
    if hours>24:
      return 0.5
    return hours




In [ ]:
def find_key(dictionary, value):

    for key, val in dictionary.items():
        if val == value:
            return key
    return -1


In [ ]:
sampling_rate = 10
homeid = '212'
appliance = 'tumbledrier'
activations = get_appliance_activations(appliance, [homeid], verbose=1)[homeid]

display(activations)

,start,end,Energy (J)
id,,,
0,2018-02-16 08:30:30,2018-02-16 08:53:10,3176752.0
1,2018-06-13 19:40:20,2018-06-13 20:05:50,1851246.0
2,2018-06-13 20:35:10,2018-06-13 20:59:40,1673604.0


In [ ]:
# activations partition

from datetime import datetime
from dateutil.relativedelta import relativedelta

timestamps1=[]
values1=[]
for i in range(len(data1)):
    # Get the necessary data from data1
    homeid = data1[i]['homeid']
    appliance = data1[i]['subtype']
    activations = get_appliance_activations(appliance, [homeid], verbose=1)[homeid]

    result_lists1, result_lists2 = create_lists_within_time_range(data1[i]['readings'],activations)
    timestamps1.append(result_lists1)
    values1.append(result_lists2)
    print(i)






0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


In [ ]:
import pickle

# Store the lists in a file
with open('timestamps1.pkl', 'wb') as f:
    pickle.dump(timestamps1, f)

with open('values1.pkl', 'wb') as f:
    pickle.dump(values1, f)

# # Reading the lists back from the file
# with open('timestamps1.pkl', 'rb') as f:
#     timestamps1 = pickle.load(f)

# with open('values1.pkl', 'rb') as f:
#     values1 = pickle.load(f)


In [ ]:
import pickle
with open('timestamps1.pkl', 'rb') as f:
    timestamps1 = pickle.load(f)


In [ ]:
with open('values1.pkl', 'rb') as f:
    values1 = pickle.load(f)


In [ ]:
timerecords1=[]
for i in range(len(data1)):
  timerecords_1={}
  for j in range(len(data1[i]['readings'].index)):
    timerecords_1[data1[i]['readings'].index[j]]=j
  timerecords1.append(timerecords_1)

In [ ]:
dates1=[]

for i in range(len(timestamps1)):
  unique_dates = set()

  for k in timestamps1[i]:
    for timestamp in k:
      date = datetime.strptime(str(timestamp), "%Y-%m-%d %H:%M:%S").date()
      unique_dates.add(date)

  num_dates = len(unique_dates)
  dates1.append(num_dates)

dates1

[150, 72, 125, 74, 193, 11, 58, 18, 74, 32, 199, 120, 221, 107, 4, 8, 146]

In [ ]:
data1[54]

{'homeid': '276',
 'roomid': '2561',
 'room_type': 'kitchen',
 'category': 'electric-appliance',
 'subtype': 'washingmachinetumbledrier',
 'sensorid': '22067',
 'readings': time
 2018-05-03 13:45:25      1
 2018-05-03 13:45:27      1
 2018-05-03 13:45:30      1
 2018-05-03 13:46:09      1
 2018-05-03 13:46:11      2
                       ... 
 2018-05-22 15:46:23    213
 2018-05-22 15:46:24    227
 2018-05-22 15:46:37    213
 2018-05-22 15:46:39    224
 2018-05-22 15:47:01     88
 Name: washingmachinetumbledrier, Length: 15767, dtype: int64}

In [ ]:
dates1[35:50]

In [ ]:
len(timestamps1[59])

118

In [ ]:
timestamps1[0][0][0].strftime('%Y-%m-%d %H:%M:%S')

'2017-08-09 15:16:33'

In [ ]:
def calculate_length_years(timestamps_list):
    if len(timestamps_list) > 0:
        first_time = timestamps_list[0][0]
        last_time = timestamps_list[-1][-1]
        length_years = (last_time - first_time).days / 365.25
        return round(length_years, 2)
    else:
        return None

In [ ]:
data1[4]

{'homeid': '276',
 'roomid': '2561',
 'room_type': 'kitchen',
 'category': 'electric-appliance',
 'subtype': 'kettle',
 'sensorid': '22075',
 'readings': time
 2018-05-03 15:32:59       0
 2018-05-03 16:32:59       0
 2018-05-03 17:33:00       0
 2018-05-03 18:33:00       0
 2018-05-03 19:33:00       1
                        ... 
 2018-05-22 13:24:06       0
 2018-05-22 14:24:06       0
 2018-05-22 15:08:02    3005
 2018-05-22 15:10:14       1
 2018-05-22 15:24:06       0
 Name: kettle, Length: 242, dtype: int64}

In [ ]:
with open('timestamps1.pkl', 'rb') as f:
    timestamps1 = pickle.load(f)


In [ ]:
# main calculation

import math
import numpy as np

import csv
from datetime import datetime, timedelta

with open('savings 2022.csv', 'a', newline='') as file:
    writer = csv.writer(file)
# 58
    for t in range(len(timestamps1)):
      info=[]
      info.append(data1[t]['homeid'])
      info.append(data1[t]['roomid'])
      info.append(data1[t]['subtype'])
      saving = 0

      timestamps = timestamps1[t]
      year=calculate_length_years(timestamps)

      values = values1[t]
      for i in range(len(timestamps)):
        if len(timestamps[i])>1:
          k = math.ceil(calculate_hours(timestamps[i][0], timestamps[i][len(timestamps[i])-1]) * 2)
          demand=[]
          price=0
          for j in range(len(timestamps[i])):
                lasttime = pd.Timestamp(find_key(timerecords1[t], timerecords1[t][timestamps[i][j]] + 1))

                hours = calculate_hours(timestamps[i][j],lasttime)
                timestamps[i][j]=timestamps[i][j].replace(year=2022)
                time = timestamps[i][j].strftime('%Y-%m-%d %H:%M:%S')
                if hours<1 and getPrice(time) is not None:
                    demand.append(values[i][j] * hours / 1000)
                    price+=values[i][j] * hours / 1000*getPrice(time)
          df['time'] = pd.to_datetime(df['time'])
          filtered_df = df[df['time'].dt.date == timestamps[i][0].date()]
#           print(timestamps[i][0].date())
          tariff = filtered_df['price'].tolist()
          if len(demand) > 0:
              demand_interval = k / len(demand)  # Time interval for demand data
          else:
            continue
          tariff_interval = 48 / len(tariff)  # Time interval for tariff data

# Resample demand to match the time interval of tariff
          resampled_demand = np.repeat(demand, int(len(tariff) * tariff_interval / demand_interval))

# Perform the convolution
          convolution_result = np.convolve(np.abs(resampled_demand), np.abs(tariff), mode='valid')
          # Set a threshold to filter out very low values
        # Set a threshold to filter out very low values
          threshold = 0.001  # You can adjust this threshold based on your data
          convolution_result_filtered = [x for x in convolution_result if x > threshold]

          if len(convolution_result_filtered) > 0:
# Find the cheapest period
            cheapest_period_index = np.argmin(convolution_result_filtered)
            cheapest_period_value = convolution_result[cheapest_period_index]
#             print(price)
#             print(cheapest_period_value)
            if  price-cheapest_period_value>0 and cheapest_period_value <10:
                  saving+=(price-cheapest_period_value)



      print(saving)
      info.append(len(timestamps))
      info.append(year)
      info.append(format(saving, '.2f'))
      writer.writerow(info)
      file.flush()

In [ ]:
for i in range(59):
    print(data1[i]['homeid'],data1[i]['roomid'],data1[i]['subtype'],len(timestamps1[i]),calculate_length_years(timestamps1[i]),sep=',')

96,999,kettle,87,0.13
73,781,kettle,439,1.32
105,1110,kettle,1621,1.1
268,2471,kettle,160,0.17
276,2561,kettle,7,0.05
90,957,kettle,919,1.22
264,2437,kettle,435,0.27
65,733,kettle,2227,1.41
61,695,kettle,1371,0.71
328,2980,kettle,25,0.02
262,2414,kettle,10,0.0
63,720,kettle,1103,1.37
231,2147,kettle,532,0.21
249,2305,kettle,373,0.14
311,2857,kettle,21,0.02
62,710,kettle,1792,1.71
259,2487,kettle,75,0.07
175,1636,kettle,854,0.74
208,1935,kettle,548,0.44
227,2103,kettle,461,0.27
136,1294,kettle,130,0.03
128,1253,kettle,489,0.88
162,1622,kettle,0,None
136,1294,kettle,25,0.01
168,1534,kettle,619,0.24
168,1534,kettle,6,0.0
146,1355,kettle,1134,0.58
106,1085,kettle,458,1.07
73,781,washingmachine,139,1.33
105,1110,washingmachine,99,1.03
106,1085,washingmachine,175,1.05
136,1294,washingmachine,37,0.85
128,1253,washingmachine,14,0.41
139,1315,washingmachine,63,0.67
175,1636,washingmachine,118,0.61
171,1565,washingmachine,64,0.54
145,1349,washingmachine,40,0.64
208,1935,washingmachine,73,0.27
22